In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython import display
from climbing_wire.homography.homography import compute_homography
from climbing_wire.homography.homography import perspective_transform
from climbing_wire.landmark.compute import PoseImg
from climbing_wire.landmark.compute import compute_landmarks
from climbing_wire.landmark.drawing import draw_landmarks
from climbing_wire.landmark.landmark_list import LandmarkListImg
from climbing_wire.utils.cv import cv_imshow
from climbing_wire.utils.cv import show_frame
from climbing_wire.utils.cv import show_warp
from climbing_wire.utils.data import get_package_fol
from climbing_wire.utils.mediapipe import JOINT_NAMES
from climbing_wire.utils.mediapipe import get_default_pose_connections
from climbing_wire.video.frame import Frame
from climbing_wire.video.load import iterate_video_frames
from climbing_wire.video.load import iterate_video_frames_with_timestamp
from climbing_wire.video.load import list_video_frames_with_timestamp
from climbing_wire.video.load import pairwise_video_frames
from loguru import logger as lg
from matplotlib import pyplot as plt
from pathlib import Path
from scipy.interpolate import splrep, BSpline
from typing import Any
from typing import List, Tuple, cast
import cv2 as cv
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles
import mediapipe.python.solutions.drawing_utils as mp_drawing
import mediapipe.python.solutions.pose as mp_pose
import numpy as np
import pathlib
import sys


In [ ]:
# from dataclasses import dataclass


# @dataclass
# class Frame:
#     frame: np.ndarray
#     usec: int
#     idx: int

#     def __str__(self) -> str:
#         return f"Frame(idx={self.idx}, usec={self.usec})"

#     # dataclasses have repr already test that
#     # def __repr__(self) -> str:
#     #     return str(self)


In [ ]:
# in_fol = Path("/mnt/c/Users/nobilip/Videos/dataset/rock")
in_fol = Path("~/data/rock").expanduser()

# person_fn = "PXL_20230401_135540281.TS.mp4"
person_fn = "andre_01.mp4"
empty_fn = "empty_01.mp4"

# in_fn = "nonaka_miho_01.mp4"

person_vid_path = in_fol / person_fn
empty_vid_path = in_fol / empty_fn


In [ ]:
keep_every_nth_frame = 15
max_num_frames = 30
fps = 30
print(
    f"will keep a frame every {keep_every_nth_frame * fps} milliseconds,"
    f" will load {fps * keep_every_nth_frame * max_num_frames} milliseconds"
)

# frame_num = 0
# for frame, usec in iterate_video_frames_with_timestamp(
#     person_vid_path,
#     keep_every_nth_frame=keep_every_nth_frame,
# ):
#     # print(usec)
#     f = Frame(frame=frame, usec=usec, idx=frame_num)
#     per_frames.append(f)
#     frame_num += 1
#     if frame_num >= max_num_frames:
#         break

# per_frames: list[Frame] = []
per_frames = list_video_frames_with_timestamp(
    person_vid_path,
    keep_every_nth_frame=keep_every_nth_frame,
    max_frame_count=max_num_frames,
)

print(f"{len(per_frames)} frames")


In [ ]:
keep_every_nth_frame = 3
# max_num_frames = 15
max_num_frames = 1500
fps = 30
print(
    f"will keep a frame every {keep_every_nth_frame * fps} milliseconds,"
    f" will load {fps * keep_every_nth_frame * max_num_frames} milliseconds"
)

# empty_frames: list[Frame] = []
# frame_num = 0
# for frame, usec in iterate_video_frames_with_timestamp(
#     empty_vid_path,
#     keep_every_nth_frame=keep_every_nth_frame,
# ):
#     # print(usec)
#     f = Frame(frame=frame, usec=usec, idx=frame_num)
#     empty_frames.append(f)
#     frame_num += 1
#     if frame_num >= max_num_frames:
#         break

empty_frames = list_video_frames_with_timestamp(
    empty_vid_path,
    keep_every_nth_frame=keep_every_nth_frame,
    max_frame_count=max_num_frames,
)

print(len(empty_frames))


In [ ]:
# def show_frame(
#     frame: Frame,
#     ax: plt.Axes | None = None,
# ):
#     """A wrapper around cv_imshow that adds a title and removes the ticks.
#     #
#     TODO: let cv_imshow return the ax it creates, so we can chain it with this function.
#     """
#     if ax is None:
#         fig, ax = plt.subplots(1, 1)
#     cv_imshow(frame.frame, ax=ax)
#     ax.set_title(f"{frame.idx} @ {frame.usec}")
#     ax_params = dict(
#         which="both",
#         # bottom=False,
#         # top=False,
#         # left=False,
#         # right=False,
#         labelbottom=False,
#         labeltop=False,
#         labelleft=False,
#         labelright=False,
#     )
#     ax.tick_params(**ax_params)


fig, ax = plt.subplots(1, 2)
# show_frame(per_frames[7], ax=ax[0])
show_frame(per_frames[-2], ax=ax[0])
# show_frame(empty_frames[11], ax=ax[1])
show_frame(empty_frames[8], ax=ax[1])


In [ ]:
# def show_warp(f1: Frame, f2: Frame, M: np.ndarray) -> None:
#     """Draw the first image warped on the second.

#     TODO: Let f1 be a Frame|np.ndarray, then extract as needed.
#     """
#     f1img = f1.frame
#     f2img = f2.frame

#     corners = np.array(
#         [
#             [0, 0],
#             [f1img.shape[1], 0],
#             [f1img.shape[1], f1img.shape[0]],
#             [0, f1img.shape[0]],
#         ],
#         dtype=np.float32,
#     )
#     corners_warp = perspective_transform(corners, M)
#     f2_ann = cv.polylines(
#         f2img.copy(), [np.int32(corners_warp)], True, 255, 3, cv.LINE_AA
#     )
#     # blend the person frame on the empty frame
#     f1_warped = cv.warpPerspective(f1img, M, f1img.shape[:2][::-1])
#     f_blend = cv.addWeighted(f1_warped, 0.5, f2_ann, 0.5, 0)

#     fig, ax = plt.subplots(1, 1)
#     ax_params = dict(
#         which="both",
#         # bottom=False,
#         # top=False,
#         # left=False,
#         # right=False,
#         labelbottom=False,
#         labeltop=False,
#         labelleft=False,
#         labelright=False,
#     )
#     ax.tick_params(**ax_params)
#     ax.set_title(f"{f1.idx} -> {f2.idx}")
#     cv_imshow(f_blend, ax=ax)

#     plt.show()


In [ ]:
# f1 = per_frames[0]
# f2 = empty_frames[0]

f1 = per_frames[7]
f2 = empty_frames[11]

# f1 = per_frames[-1]
# f2 = empty_frames[-1]


def homography_distance(
    f1: Frame,
    f2: Frame,
    do_plot: bool = False,
    ax: plt.Axes | None = None,
) -> float:
    # lg.debug(f"{f1.usec=} {f2.usec=}")
    # print(".", end="", flush=True)

    f1img = f1.frame
    f2img = f2.frame

    # shrink images to speed up computation
    f1img = cv.resize(f1img, (0, 0), fx=0.5, fy=0.5)
    f2img = cv.resize(f2img, (0, 0), fx=0.5, fy=0.5)

    # compute homography might fail if there are not enough features
    # so we just return a large number
    try:
        M = compute_homography(f1img, f2img)
    except ValueError:
        lg.debug(f"failed homography for {f1} {f2}")
        # assume that one image is completely outside the other
        # return (f1img.shape[0] + f1img.shape[1]) * 2
        return ((f1img.shape[0] + f1img.shape[1]) * 2) ** 2

    corners = np.array(
        [
            [0, 0],
            [f1img.shape[1], 0],
            [f1img.shape[1], f1img.shape[0]],
            [0, f1img.shape[0]],
        ],
        dtype=np.float32,
    )
    corners_warp = perspective_transform(corners, M)
    corners_delta = corners_warp - corners
    # dist = np.linalg.norm(corners_delta, axis=1).mean()
    dist = (corners_delta**2).mean()

    # draw the first image warped on the second
    # rather than doing this, turn homography_distance into a class
    # maybe a single class with class methods?
    if do_plot:
        show_warp(
            Frame(f1img, f1.usec, f1.idx),
            Frame(f2img, f2.usec, f2.idx),
            M,
            ax=ax,
            dist=dist,
        )

    return dist


homography_distance(f1, f2, do_plot=True)
# plt.show()


In [ ]:
# # draw the warped corners on the image
# f2_ann = cv.polylines(
#     f2.frame.copy(), [np.int32(corners_warp)], True, 255, 3, cv.LINE_AA
# )
# # blend the person frame on the empty frame
# f1_warped = cv.warpPerspective(f1.frame, M, f1.frame.shape[:2][::-1])
# f_blend = cv.addWeighted(f1_warped, 0.5, f2_ann, 0.5, 0)
# cv_imshow(f_blend)


In [ ]:
len(per_frames), len(empty_frames)


#### Compute path with DTW


In [ ]:
from fastdtw.fastdtw import fastdtw

# from fastdtw.fastdtw import __dtw
# from fastdtw.fastdtw import __fastdtw

distance, path = fastdtw(
    # distance, path = __fastdtw(
    per_frames[:-2],
    empty_frames[:],
    10,
    dist=homography_distance,
)
print(distance, path)


In [ ]:
# plt.plot([x[0] for x in path], [x[1] for x in path])
plt.scatter([x[0] for x in path], [x[1] for x in path])


#### Interpolate with pandas


In [ ]:
import pandas as pd

pathdf = pd.DataFrame(
    path,
    columns=["per_idx", "empty_idx"],
)
pathdf["per_usec"] = pathdf["per_idx"].apply(lambda idx: per_frames[idx].usec)
pathdf["empty_usec"] = pathdf["empty_idx"].apply(lambda idx: empty_frames[idx].usec)
pathdf.head(20)


In [ ]:
def interpolate_empty_usec(
    pathdf: pd.DataFrame,
    new_per_usec: int,
) -> int:
    """Linear interpolation of empty_usec given per_usec."""
    per_usec_before = pathdf[pathdf["per_usec"] <= new_per_usec]["per_usec"].max()
    per_usec_after = pathdf[pathdf["per_usec"] > new_per_usec]["per_usec"].min()
    print(per_usec_before, per_usec_after)

    em_usec_before, em_usec_after = pathdf[
        pathdf["per_usec"].isin([per_usec_before, per_usec_after])
    ]["empty_usec"].values
    print(em_usec_before, em_usec_after)

    # interpolate the empty_usec value
    em_usec_new = em_usec_before + (new_per_usec - per_usec_before) * (
        em_usec_after - em_usec_before
    ) / (per_usec_after - per_usec_before)
    print(em_usec_new)
    return int(em_usec_new)


new_per_usec = 1_000_000
interpolate_empty_usec(pathdf, new_per_usec)


In [ ]:
# # show the path frame warps
# for per_idx, empty_idx in path:
#     f1 = per_frames[per_idx]
#     f2 = empty_frames[empty_idx]
#     homography_distance(f1, f2, do_plot=True)


#### Interpolate with scipy


In [ ]:
# path to frame pairs
path_frames: list[tuple[Frame, Frame]] = [
    (per_frames[p[0]], empty_frames[p[1]]) for p in path
]
path_frames[3]


In [ ]:
# extract idx
per_idx_path = [f[0].idx for f in path_frames]
empty_idx_path = [f[1].idx for f in path_frames]

# extract usec
per_usec_path = [f[0].usec for f in path_frames]
empty_usec_path = [f[1].usec for f in path_frames]


In [ ]:
def splrep_(per_usec_path, empty_usec_path, **kwargs) -> tuple[Any, Any, int]:
    tck = cast(tuple[Any, Any, int], splrep(per_usec_path, empty_usec_path, **kwargs))
    return tck


tck = splrep_(per_usec_path, empty_usec_path)
xnew = np.linspace(0, max(per_usec_path), 100)
plt.scatter(per_usec_path, empty_usec_path)
plt.plot(xnew, BSpline(*tck)(xnew))


In [ ]:
def interpolate_spline(tck, x):
    """Interpolate a spline at a given point."""
    return BSpline(*tck)([x])[0]


requested_usec = 1000000
interpolate_spline(tck, requested_usec)


In [ ]:
# now we load the whole empty video
# TODO turn it into a coroutine so that we can change the target millisecond frame


In [ ]:
keep_every_nth_frame = 1
max_num_frames = 1500
fps = 30
print(
    f"will keep a frame every {keep_every_nth_frame * fps} milliseconds,"
    f" will load {fps * keep_every_nth_frame * max_num_frames} milliseconds"
)

# empty_frames_all: list[Frame] = []
# frame_num = 0
# for frame, usec in iterate_video_frames_with_timestamp(
#     empty_vid_path,
#     keep_every_nth_frame=keep_every_nth_frame,
# ):
#     f = Frame(frame=frame, usec=usec, idx=frame_num)
#     empty_frames_all.append(f)
#     frame_num += 1
#     if frame_num >= max_num_frames:
#         break

empty_frames_all = list_video_frames_with_timestamp(
    empty_vid_path,
    keep_every_nth_frame=keep_every_nth_frame,
    max_frame_count=max_num_frames,
)

print(len(empty_frames_all))


In [ ]:
# get the frame closest to a given usec


def get_closest_frame(
    frames: list[Frame],
    usec: int,
) -> Frame:
    """Get the frame closest to a given usec."""
    return min(frames, key=lambda f: abs(f.usec - usec))


get_closest_frame(empty_frames_all, 1000000)


In [ ]:
def warp_landlist(
    landlist: LandmarkListImg,
    M: np.ndarray,
) -> LandmarkListImg:
    """Warp a landmark list.

    Note that this does not change the normalized landmarks.
    """
    lli_warped = landlist.copy()
    lli_warped.landmarks_img = perspective_transform(
        lli_warped.landmarks_img.astype(np.float64), M
    ).astype(int)
    return lli_warped


In [ ]:
# load the whole person video
# find the closest frame to the empty frame by interpolation
# find the wireframe of the person
# compute the homography between the person and the empty frame
# warp it to the empty frame
# plot it on the empty frame
# save the empty frame

from tqdm import tqdm


cds = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=8, circle_radius=2)
lds = mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=8, circle_radius=8)

pose_img_kwargs: dict[str, Any] = {}
pose_img = PoseImg(**pose_img_kwargs)


keep_every_nth_frame = 1
max_num_frames = 500
fps = 30
print(
    f"will keep a frame every {keep_every_nth_frame * fps} milliseconds,"
    f" will load {fps * keep_every_nth_frame * max_num_frames} milliseconds"
)

frame_width = 1080
frame_height = 1920
print("Frame size ", (frame_width, frame_height))
output = cv.VideoWriter(
    "output_video.mp4",
    cv.VideoWriter_fourcc(*"mp4v"),
    30,
    (int(frame_width), int(frame_height)),
)

pbar = tqdm(total=max_num_frames)

frame_num = 0
for frame, usec in iterate_video_frames_with_timestamp(
    person_vid_path,
    keep_every_nth_frame=keep_every_nth_frame,
):
    # print(usec)
    per_frame = Frame(frame=frame, usec=usec, idx=frame_num)

    # find the wireframe of the person
    wireframe = pose_img(per_frame.frame)
    if wireframe is None:
        continue

    # find the closest empty frame
    closest_empty_frame = get_closest_frame(empty_frames_all, per_frame.usec)

    # compute the homography between the person and the empty frame
    try:
        M = compute_homography(per_frame.frame, closest_empty_frame.frame)
    except ValueError:
        lg.debug(f"Could not compute homography {per_frame} {closest_empty_frame}")
        continue

    lli_warped = warp_landlist(wireframe, M)
    ef_ann = closest_empty_frame.frame.copy()
    draw_landmarks(
        ef_ann,
        lli_warped,
        landmark_drawing_spec=lds,
        connection_drawing_spec=cds,
    )
    # cv_imshow(ef_ann)
    output.write(ef_ann)

    pbar.update(1)

    frame_num += 1
    if frame_num >= max_num_frames:
        break

    # break

pbar.close()
# pose_img.close()
output.release()


In [ ]:
# experiments with warping a wireframe
# cds = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=8, circle_radius=2)
# lds = mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=8, circle_radius=8)
# lli_warped = warp_landlist(wireframe, M)
# ef_ann = closest_empty_frame.frame.copy()
# draw_landmarks(
#     ef_ann,
#     lli_warped,
#     landmark_drawing_spec=lds,
#     connection_drawing_spec=cds,
# )
# cv_imshow(ef_ann)


In [ ]:
# # create video from frames using opencv
# frame_width = 1080
# frame_height = 1920
# print("Frame size ", (frame_width, frame_height))
# output = cv.VideoWriter(
#     "output_video.mp4",
#     cv.VideoWriter_fourcc(*"mp4v"),
#     30,
#     (int(frame_width), int(frame_height)),
# )
# output.write(annotated_frame)
# output.release()


## Neat but we want a better test of DTW


In [ ]:
# load a lot of frames

keep_every_nth_frame = 5
max_num_frames = 60
fps = 30
print(
    f"will keep a frame every {keep_every_nth_frame * fps} milliseconds,"
    f" will load {fps * keep_every_nth_frame * max_num_frames} milliseconds"
)

# many_frames: list[Frame] = []
# frame_num = 0
# for frame, usec in iterate_video_frames_with_timestamp(
#     person_vid_path,
#     keep_every_nth_frame=keep_every_nth_frame,
# ):
#     # print(usec)
#     f = Frame(frame=frame, usec=usec, idx=frame_num)
#     many_frames.append(f)
#     frame_num += 1
#     if frame_num >= max_num_frames:
#         break

many_frames = list_video_frames_with_timestamp(
    person_vid_path,
    keep_every_nth_frame=keep_every_nth_frame,
    max_frame_count=max_num_frames,
)
print(f"{len(many_frames)} frames")


In [ ]:
vid1_frames_idx = [
    0,
    4,
    8,
    14,
    19,
    23,
    29,
    31,
    36,
    39,
    40,
    41,
    42,
    43,
    48,
    55,
    58,
]

vid2_frames_idx = [
    2,
    3,
    7,
    17,
    19,
    24,
    28,
    32,
    37,
    38,
    45,
    48,
    58,
]

vid1_frames = [f for f in many_frames if f.idx in vid1_frames_idx]
vid2_frames = [f for f in many_frames if f.idx in vid2_frames_idx]


In [ ]:
distance, path = fastdtw(
    vid1_frames,
    vid2_frames,
    10,
    dist=homography_distance,
)
print(distance, path)


In [ ]:
x = [p[0] for p in path]
y = [p[1] for p in path]
plt.scatter(x, y)


In [ ]:
# https://docs.scipy.org/doc/scipy/tutorial/interpolate/smoothing_splines.html


In [ ]:
# A tuple (t,c,k) containing the vector of knots,
# the B-spline coefficients, and the degree of the spline.

# set smoothing to 1
tck_idx: tuple[Any, Any, int] = splrep(x, y, s=1)
xnew = np.linspace(0, max(x), 100)
plt.scatter(x, y)
plt.plot(xnew, BSpline(*tck_idx)(xnew))


In [ ]:
# see the matched frame idx
# the match here should be 1:1
path1idx = [vid1_frames[p[0]].idx for p in path]
path2idx = [vid2_frames[p[1]].idx for p in path]
xend = max(path1idx)

tck_idx: tuple[Any, Any, int] = splrep(path1idx, path2idx, s=1)
xnew = np.linspace(0, max(path1idx), 100)
ynew = BSpline(*tck_idx)(xnew)
plt.plot(xnew, ynew)

plt.scatter(path1idx, path2idx)
plt.plot([0, xend], [0, xend], color="red")


In [ ]:
# path to frame pairs
path_frames = [(vid1_frames[p[0]], vid2_frames[p[1]]) for p in path]


In [ ]:
path_frames[0][0].frame.shape


## Compute the whole distance matrix


In [ ]:
# load a lot of frames

keep_every_nth_frame = 5
max_num_frames = 60
fps = 30
print(
    f"will keep a frame every {keep_every_nth_frame * fps} milliseconds,"
    f" will load {fps * keep_every_nth_frame * max_num_frames} milliseconds"
)

many_frames = list_video_frames_with_timestamp(
    person_vid_path,
    keep_every_nth_frame=keep_every_nth_frame,
    max_frame_count=max_num_frames,
)
print(f"{len(many_frames)} frames")


In [ ]:
vid1_frames_idx = [
    0,
    4,
    8,
    14,
    19,
    23,
    29,
    31,
    36,
    39,
    40,
    41,
    42,
    43,
    48,
    55,
    58,
]

vid2_frames_idx = [
    2,
    4,
    7,
    17,
    19,
    24,
    28,
    32,
    37,
    38,
    45,
    48,
    58,
]

vid1_frames = [f for f in many_frames if f.idx in vid1_frames_idx]
vid2_frames = [f for f in many_frames if f.idx in vid2_frames_idx]


In [ ]:
# use the two videos
vid1_frames = per_frames[:24:2]
vid2_frames = empty_frames[:40:2]
vid1_frames_idx = [f.idx for f in vid1_frames]
vid2_frames_idx = [f.idx for f in vid2_frames]
len(vid1_frames), len(vid2_frames)


In [ ]:
dist = np.zeros((len(vid1_frames), len(vid2_frames)))
fig, axes = plt.subplots(len(vid1_frames), len(vid2_frames), figsize=(34, 24))
for i in range(len(vid1_frames)):
    for j in range(len(vid2_frames)):
        ax = axes[i][j]
        dist[i, j] = homography_distance(
            vid1_frames[i],
            vid2_frames[j],
            do_plot=True,
            ax=ax,
        )

fig.tight_layout()
plt.show()
plt.close(fig)


In [ ]:
fig, ax = plt.subplots(1, 1)
ax.imshow(dist)
ax.set_xticks(np.arange(len(vid2_frames)))
ax.set_yticks(np.arange(len(vid1_frames)))
ax.set_xticklabels(vid2_frames_idx)
ax.set_yticklabels(vid1_frames_idx)

plt.show()
plt.close(fig)


In [ ]:
dist
